In [ ]:
!pip install pyreadstat

In [1]:
import pyreadstat
import os
import pandas as pd
import openpyxl
import numpy as np

from sklearn.model_selection import train_test_split

# 1. Data Extraction

In [2]:
columns = {
    "ALQ_L.XPT": ["SEQN", "ALQ121"],
    "BMX_L.XPT": ["SEQN", "BMXWT", "BMXHT", "BMXBMI"],
    "BPQ_L.XPT": ["SEQN", "BPQ020"],
    "BPXO_L.XPT": ["SEQN", "BPXOSY1","BPXODI1","BPXOPLS1"],
    "CBC_L.XPT": ["SEQN","LBXHGB"],
    "DEMO_L.XPT": ["SEQN", "RIAGENDR","RIDAGEYR","DMDBORN4","DMDEDUC2","DMDMARTZ","RIDEXPRG","DMDHHSIZ"],
    "DIQ_L.XPT": ["SEQN", "DIQ010"],
    "DPQ_L.XPT": ["SEQN", "DPQ010","DPQ020","DPQ030","DPQ040","DPQ050","DPQ060","DPQ070","DPQ080","DPQ090"],
    "FNQ_L.XPT": ["SEQN", "FNQ410","FNQ430","FNQ440","FNQ450","FNQ460","FNQ470","FNQ490"],
    "FOLATE_L.XPT": ["SEQN", "LBDRFOSI"],
    "HDL_L.XPT": ["SEQN", "LBDHDDSI"],
    "HIQ_L.XPT": ["SEQN", "HIQ011"],
    "HOQ_L.XPT": ["SEQN", "HOD051"],
    "HSCRP_L.XPT": ["SEQN", "LBXHSCRP"],
    "HUQ_L.XPT": ["SEQN", "HUQ010","HUQ030","HUQ090"],
    "INQ_L.XPT": ["SEQN", "INDFMMPI","INQ300"],
    "MCQ_L.XPT": ["SEQN","MCQ160B","MCQ160E","MCQ160F","MCQ160P","MCQ220"],
    "OCQ_L.XPT": ["SEQN", "OCD150","OCQ180","OCQ215"],
    "OHQ_L.XPT": ["SEQN", "OHQ845","OHQ680"],
    "PAQ_L.XPT": ["SEQN", "PAD790Q","PAD790U","PAD680"],
    "PBCD_L.XPT": ["SEQN", "LBXBPB","LBXTHG"],
    "SLQ_L.XPT": ["SEQN", "SLD012","SLD013"],
    "VID_L.XPT": ["SEQN", "LBXVD2MS"],
    "WHQ_L.XPT": ["SEQN", "WHQ070"],
}

In [3]:
all_dfs = []

for filename, selected_columns in columns.items():
    file_path = os.path.join("Raw_Datasets", filename)
    df, meta = pyreadstat.read_xport(file_path, encoding='cp1252')
    df = df[selected_columns]
    all_dfs.append(df)
    
merged_df = all_dfs[0]
for df in all_dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="SEQN", how="outer")

# Drop rows where all specified columns are empty
columns_to_check = ["DPQ010", "DPQ020", "DPQ030", "DPQ040", "DPQ050", "DPQ060", "DPQ070", "DPQ080", "DPQ090"]
merged_df = merged_df.dropna(subset=columns_to_check, how='all')

merged_df.to_csv("merged_nhanes.csv", index=False)

# 2. Data Cleaning

In [4]:
dict_df = pd.read_csv("dictionary.csv")
lookup = dict(zip(dict_df['var_nhanes'], dict_df['var_id']))
merged_df = merged_df.rename(columns=lookup)

In [5]:
merged_df.shape

(5519, 63)

In [6]:
print(merged_df.columns)

Index(['SEQN', 'habit01', 'physical01', 'physical02', 'physical03', 'com02',
       'physical04', 'physical05', 'physical06', 'lab04', 'demo01', 'demo02',
       'demo03', 'demo05', 'demo06', 'demo07', 'demo08', 'com03', 'target01',
       'target02', 'target03', 'target04', 'target05', 'target06', 'target07',
       'target08', 'target09', 'func01', 'func02', 'func03', 'func04',
       'func05', 'func06', 'func07', 'lab06', 'lab08', 'healthcare01',
       'demo10', 'lab10', 'com01', 'healthcare02', 'healthcare04', 'demo11',
       'demo12', 'com07', 'com08', 'com09', 'com10', 'com13', 'job01', 'job02',
       'job04', 'com15', 'com16', 'habit02', 'habit03', 'habit04', 'lab11',
       'lab12', 'habit05', 'habit06', 'lab14', 'habit08'],
      dtype='object')


In [7]:
columns1 = ["target01", "target02", "target03", "target04", "target05", "target06", "target07", "target08", "target09"]
for column_1 in columns1:
    merged_df[column_1] = merged_df[column_1].replace(7, 0).replace(9, 0).fillna(0)

columns2 = ['com02', 'com07', 'com08', 'com09', 'com10', 'com13', 'demo12', 'habit08',
            'healthcare01', 'healthcare02', 'healthcare04', 'demo03']
for column_2 in columns2:
    merged_df[column_2] = merged_df[column_2].replace([2, 7, 9], 0).fillna(0)

columns3 = ['com01', 'com15']
for column_3 in columns3:
    merged_df[column_3] = merged_df[column_3].replace([7, 9], 2).fillna(2)

columns4 = ['com03']
for column_4 in columns4:
    merged_df[column_4] = merged_df[column_4].replace([2, 3, 7, 9], 0).fillna(0)

columns5 = ['com16']
for column_5 in columns5:
    merged_df[column_5] = merged_df[column_5].replace([7, 9], 4).fillna(1)

columns6 = ['demo05']
for column_6 in columns6:
    merged_df[column_6] = merged_df[column_6].replace([7, 9], 4).fillna(4)

columns8 = ['func01', 'func02', 'func03', 'func04', 'func05', 'func06', 'func07']
for column_8 in columns8:
    merged_df[column_8] = merged_df[column_8].replace([7, 9], 1).fillna(1)

columns10 = ['demo06']
for column_10 in columns10:
    merged_df[column_10] = merged_df[column_10].replace([77, 99], 3).fillna(3)

columns11 = ['demo10']
for column_11 in columns11:
    merged_df[column_11] = merged_df[column_11].replace([77, 99], 5).fillna(5)

columns12 = ['demo08']
for column_12 in columns12:
    merged_df[column_12] = merged_df[column_12].fillna(2)

columns13 = ['demo07']
for column_13 in columns13:
    merged_df[column_13] = merged_df[column_13].replace([2, 3], 0).fillna(0)

columns14 = ['habit01']
for column_14 in columns14:
    merged_df[column_14] = merged_df[column_14].replace({77: 5, 99: 5, 0: 11}).fillna(5)

columns15 = ['habit02', 'habit04']
for column_15 in columns15:
    merged_df[column_15] = merged_df[column_15].replace([7777, 9999], 0).fillna(0)

columns16 = ['job04']
for column_16 in columns16:
    merged_df[column_16] = merged_df[column_16].replace([77, 99], 0).fillna(0)

columns17 = ['job01']
for column_17 in columns17:
    merged_df[column_17] = merged_df[column_17].replace({3: 0, 4: 0, 7: 0, 9: 0, 2: 1}).fillna(0)

columns18 = ['job02']
for column_18 in columns18:
    merged_df[column_18] = merged_df[column_18].replace([77777, 99999], 0).fillna(0)

merged_df['demo11'] = merged_df['demo11'].fillna(2.78)
merged_df['habit05'] = merged_df['habit05'].fillna(7.7)
merged_df['habit06'] = merged_df['habit06'].fillna(8.3)
merged_df['lab06'] = merged_df['lab06'].fillna(1216)
merged_df['lab08'] = merged_df['lab08'].fillna(1.41)
merged_df['lab10'] = merged_df['lab10'].fillna(3.79)
merged_df['lab11'] = merged_df['lab11'].fillna(0.956)
merged_df['lab12'] = merged_df['lab12'].fillna(1.16)
merged_df['lab14'] = merged_df['lab14'].fillna(4.8)

merged_df['habit03'] = merged_df['habit03'].replace({"D": 365, "M": 12, "W": 52, "Y": 1})
merged_df['habit03'] = merged_df['habit03'].where(merged_df.habit02 != 0, other=0)
merged_df['habit09'] = merged_df.habit02 * merged_df.habit03
merged_df = merged_df.drop(['habit02', 'habit03'], axis=1)

merged_df.lab04 = np.where(merged_df.demo01 == 1, merged_df.lab04.fillna(14.8), merged_df.lab04.fillna(13.2))
merged_df.physical01 = np.where(merged_df.demo01 == 1, merged_df.physical01.fillna(89.4), merged_df.physical01.fillna(77.9))
merged_df.physical02 = np.where(merged_df.demo01 == 1, merged_df.physical02.fillna(174.8), merged_df.physical02.fillna(161.1))
merged_df.physical03 = np.where(merged_df.demo01 == 1, merged_df.physical03.fillna(29.2), merged_df.physical03.fillna(30.0))
merged_df.physical04 = np.where(merged_df.demo01 == 1, merged_df.physical04.fillna(125), merged_df.physical04.fillna(120))
merged_df.physical05 = np.where(merged_df.demo01 == 1, merged_df.physical05.fillna(75), merged_df.physical05.fillna(74))
merged_df.physical06 = np.where(merged_df.demo01 == 1, merged_df.physical06.fillna(69), merged_df.physical06.fillna(72))



In [8]:
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)
merged_df.to_csv("final_merged.csv", index=False)

# 3. Training, Validation, Test Split

In [9]:
SEED = 77

In [10]:
merged_df = pd.read_csv("final_merged.csv")

In [11]:
merged_df['phq_sum'] = merged_df[["target01","target02","target03","target04","target05",
                                  "target06","target07","target08","target09"]].sum(axis=1)
merged_df = merged_df.drop(columns = ["SEQN","target01","target02","target03","target04","target05",
                                      "target06","target07","target08","target09"]).iloc[:, 1:]

In [13]:
merged_df.to_csv("final_merged.csv")

In [14]:
merged_df.iloc[:,-1:]

,phq_sum
0,1.0
1,2.0
2,1.0
3,0.0
4,0.0
...,...
5514,0.0
5515,0.0
5516,0.0
5517,0.0


In [15]:
# Split to 60-40 (training dataset and the rest)
input_train, input_rest, output_train, output_rest = train_test_split (merged_df.iloc[:,:-1], 
                                                                       merged_df.iloc[:,-1:], 
                                                                       test_size=0.4, 
                                                                       random_state=SEED, 
                                                                       shuffle=True)
    
# Split to 20-20 (validation dataset and test dataset)
input_validate, input_test, output_validate, output_test = train_test_split (input_rest, 
                                                                             output_rest, 
                                                                             test_size=0.5, 
                                                                             random_state=SEED, 
                                                                             shuffle=True)

In [16]:
input_train.to_csv("Training_Validation_Test_Datasets/task2_input_train.csv", index=False)
input_validate.to_csv("Training_Validation_Test_Datasets/task2_input_validate.csv", index=False)
input_test.to_csv("Training_Validation_Test_Datasets/task2_input_test.csv", index=False)

output_train.to_csv("Training_Validation_Test_Datasets/task2_output_train.csv", index=False)
output_validate.to_csv("Training_Validation_Test_Datasets/task2_output_validate.csv", index=False)
output_test.to_csv("Training_Validation_Test_Datasets/task2_output_test.csv", index=False)

# 4. Data Grouping

In [17]:
merged_df = pd.read_csv("final_merged.csv")

In [18]:
print(merged_df.columns)

Index(['Unnamed: 0', 'com02', 'com03', 'com07', 'com08', 'com09', 'com10',
       'com13', 'com15', 'com16', 'demo01', 'demo02', 'demo03', 'demo05',
       'demo06', 'demo07', 'demo08', 'demo10', 'demo11', 'demo12', 'func01',
       'func02', 'func03', 'func04', 'func05', 'func06', 'func07', 'habit01',
       'habit04', 'habit05', 'habit06', 'habit08', 'habit09', 'healthcare01',
       'healthcare02', 'healthcare04', 'job01', 'job02', 'job04', 'lab04',
       'lab06', 'lab08', 'lab10', 'lab11', 'lab12', 'lab14', 'physical01',
       'physical02', 'physical03', 'physical04', 'physical05', 'physical06',
       'phq_sum'],
      dtype='object')


In [19]:
merged_df.shape

(5519, 53)

In [20]:
conditions = {
        "Male_Younger": (merged_df['demo01'] == 1) & (merged_df['demo02'] >= 18) & (merged_df['demo02'] <= 55),
        "Male_Older": (merged_df['demo01'] == 1) & (merged_df['demo02'] >= 56),
        "Female_Younger": (merged_df['demo01'] == 2) & (merged_df['demo02'] >= 18) & (merged_df['demo02'] <= 55),
        "Female_Older": (merged_df['demo01'] == 2) & (merged_df['demo02'] >= 56)
    }

In [21]:
for group_name, condition in conditions.items():
    group_data = merged_df[condition]
    group_size = len(group_data)
    print(f"{group_name}: {group_size} entries")
    
    output_file = os.path.join("Grouping_Datasets", f"{group_name}.csv")
    group_data.to_csv(output_file, index=False)
    print(f"Saved {group_name} group data to {output_file}")

Male_Younger: 1227 entries
Saved Male_Younger group data to Grouping_Datasets/Male_Younger.csv
Male_Older: 1295 entries
Saved Male_Older group data to Grouping_Datasets/Male_Older.csv
Female_Younger: 1473 entries
Saved Female_Younger group data to Grouping_Datasets/Female_Younger.csv
Female_Older: 1524 entries
Saved Female_Older group data to Grouping_Datasets/Female_Older.csv
